<a href="https://colab.research.google.com/github/dalasjoe-1/dfl/blob/master/GetLotto_210809.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Another Way

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import math
from os import path
from google.colab import drive

tf.compat.v1.reset_default_graph()

tf.compat.v1.disable_eager_execution()

tf.compat.v1.set_random_seed(777)

input_data_column_cnt = 45  # 입력데이터의 컬럼 개수(Variable 개수)
#output_data_column_cnt = 1  # 결과데이터의 컬럼 개수
output_data_column_cnt = 45  # 결과데이터의 컬럼 개수

seq_length = 46 #192 #24 #96  # 1개 시퀀스의 길이(시계열데이터 입력 개수)
rnn_cell_hidden_dim = 20  # 각 셀의 (hidden)출력 크기
forget_bias = 1.0  # 망각편향(기본값 1.0)
num_stacked_layers = 1  # stacked LSTM layers 개수
keep_prob = 1.0  # dropout할 때 keep할 비율

epoch_num = 18000 #20000  # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01  # 학습률

# Standardization
def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()

# 너무 작거나 너무 큰 값이 학습을 방해하는 것을 방지하고자 정규화한다
# x가 양수라는 가정하에 최소값과 최대값을 이용하여 0~1사이의 값으로 변환
# Min-Max scaling
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7)  # 1e-7은 0으로 나누는 오류 예방차원

# 정규화된 값을 원래의 값으로 되돌린다
# 정규화하기 이전의 org_x값과 되돌리고 싶은 x를 입력하면 역정규화된 값을 리턴한다
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()

# data
drive.mount('/content/gdrive',force_remount=True)
#input = np.loadtxt('./gdrive/MyDrive/ori.data', unpack=True, dtype='int')
input = np.loadtxt("./gdrive/MyDrive/lotto_data.txt", delimiter=",", dtype='int')
#data = np.transpose(input)
data = input
#win_numbers = data[:,1:data.size]
win_numbers = data[:,1:7]

#print(win_numbers, " : ", win_numbers.size)
norm_win_numbers = min_max_scaling(win_numbers)  # 가격형태 데이터 정규화 처리

#x = min_max_scaling(win_numbers);
x = win_numbers
y = x[:, :]
z = []

for i in range(len(data)):
    week = win_numbers[i]  
    buckets = [0, 0, 0, 0, 0, 0, 0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0]
    for j in range(len(week)):
        k = week[j]
        buckets[k-1] = 100
    z.append(buckets)

x = z;
y = x;

dataX = []  # 입력으로 사용될 Sequence Data
dataY = []  # 출력(타켓)으로 사용

for i in range(0, len(y) - seq_length):
    _x = x[i: i + seq_length]
    _y = y[i + seq_length]  # 다음 나타날 주가(정답)
    # if i is 0:
    #      print(">>> ", _x, "->", _y)  # 첫번째 행만 출력해 봄

    dataX.append(_x)  # dataX 리스트에 추가
    dataY.append(_y)  # dataY 리스트에 추가

# 학습용/테스트용 데이터 생성
# 전체 70%를 학습용 데이터로 사용
train_size = int(len(dataY) * 0.7)
# 나머지(30%)를 테스트용 데이터로 사용
test_size = len(dataY) - train_size


# 데이터를 잘라 학습용 데이터 생성
trainX = np.asarray(dataX[0:train_size])
trainY = np.asarray(dataY[0:train_size])

# for i in range(len(trainX)):
#     aaa = trainX[i]
#     if (i==0):

print(trainX.shape, " ==== ", trainY.shape)
# print(trainX[0], " **** ", dataX[0])


# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

# 텐서플로우 플레이스홀더 생성
# 입력 X, 출력 Y를 생성한다
X = tf.compat.v1.placeholder(tf.float32, [None, seq_length, input_data_column_cnt])
print("X: ", X)
Y = tf.compat.v1.placeholder(tf.float32, [None, output_data_column_cnt])
print("Y: ", Y)

# 검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다
targets = tf.compat.v1.placeholder(tf.float32, [None, output_data_column_cnt])
print("targets: ", targets)

predictions = tf.compat.v1.placeholder(tf.float32, [None, output_data_column_cnt])
print("predictions: ", predictions)

def lstm_cell():
    # LSTM셀을 생성
    # num_units: 각 Cell 출력 크기
    # forget_bias:  to the biases of the forget gate
    #              (default: 1)  in order to reduce the scale of forgetting in the beginning of the training.
    # state_is_tuple: True ==> accepted and returned states are 2-tuples of the c_state and m_state.
    # state_is_tuple: False ==> they are concatenated along the column axis.

    cell = tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=rnn_cell_hidden_dim,
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
   
    # cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim,
    #                                     forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    #cell = tf.keras.layers.LSTMCell(num_units=rnn_cell_hidden_dim, forget_bias=forget_bias
    #                                 , state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        #cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        cell = tf.compat.v1.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell


# num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

hypothesis, _states = tf.compat.v1.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
#hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)

# [:, -1]를 잘 살펴보자. LSTM RNN의 마지막 (hidden)출력만을 사용했다.
# 과거 여러 거래일의 주가를 이용해서 다음날의 주가 1개를 예측하기때문에 MANY-TO-ONE형태이다
#hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], output_data_column_cnt, activation_fn=tf.identity)
hypothesis = tf.compat.v1.layers.dense(hypothesis[:, -1], output_data_column_cnt, activation = tf.identity)



# 손실함수로 평균제곱오차를 사용한다
loss = tf.reduce_sum(tf.square(hypothesis - Y))
# 최적화함수로 AdamOptimizer를 사용한다
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.RMSPropOptimizer(learning_rate) # LSTM과 궁합 별로임

train = optimizer.minimize(loss)

# RMSE(Root Mean Square Error)
# 제곱오차의 평균을 구하고 다시 제곱근을 구하면 평균 오차가 나온다
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions))) # 아래 코드와 같다
rmse = tf.sqrt(tf.reduce_mean(tf.math.squared_difference(targets, predictions)))

train_error_summary = []  # 학습용 데이터의 오류를 중간 중간 기록한다
test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
test_predict = ''  # 테스트용데이터로 예측한 결과

sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

print("trainX.shape=", trainX.shape, "trainY.shape=", trainY.shape)
print("trainX=", trainX)

# 학습한다
start_time = datetime.datetime.now()  # 시작시간을 기록한다
print('학습을 시작합니다...')
for epoch in range(epoch_num):
    _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    if ((epoch + 1) % 100 == 0) or (epoch == epoch_num - 1):  # 100번째마다 또는 마지막 epoch인 경우
        # 학습용데이터로 rmse오차를 구한다
        train_predict = sess.run(hypothesis, feed_dict={X: trainX})
        train_error = sess.run(rmse, feed_dict={targets: trainY, predictions: train_predict})
        train_error_summary.append(train_error)

        # 테스트용데이터로 rmse오차를 구한다
        test_predict = sess.run(hypothesis, feed_dict={X: testX})
        test_error = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        test_error_summary.append(test_error)

        # 현재 오류를 출력한다
        print("epoch: {}, train_error(A): {}, test_error(B): {}, B-A: {}".format(epoch + 1, train_error, test_error,
                                                                                 test_error - train_error))

# end_time = datetime.datetime.now()  # 종료시간을 기록한다
# elapsed_time = end_time - start_time  # 경과시간을 구한다
# print('elapsed_time:', elapsed_time)
# print('elapsed_time per epoch:', elapsed_time / epoch_num)
#
# # 하이퍼파라미터 출력
# print('input_data_column_cnt:', input_data_column_cnt, end='')
# print(',output_data_column_cnt:', output_data_column_cnt, end='')
#
# print(',seq_length:', seq_length, end='')
# print(',rnn_cell_hidden_dim:', rnn_cell_hidden_dim, end='')
# print(',forget_bias:', forget_bias, end='')
# print(',num_stacked_layers:', num_stacked_layers, end='')
# print(',keep_prob:', keep_prob, end='')
#
# print(',epoch_num:', epoch_num, end='')
# print(',learning_rate:', learning_rate, end='')
#
# print(',train_error:', train_error_summary[-1], end='')
# print(',test_error:', test_error_summary[-1], end='')
# print(',min_test_error:', np.min(test_error_summary))
#
# # 결과 그래프 출력
# plt.figure(1)
# plt.plot(train_error_summary, 'gold')
# plt.plot(test_error_summary, 'b')
# plt.xlabel('Epoch(x100)')
# plt.ylabel('Root Mean Square Error')
#
# plt.figure(2)
# plt.plot(testY, 'r')
# plt.plot(test_predict, 'b')
# plt.xlabel('Time Period')
# plt.ylabel('Stock Price')
# plt.show()

# sequence length만큼의 가장 최근 데이터를 슬라이싱한다
recent_data = np.array([x[len(x) - seq_length:]])
print("recent_data.shape:", recent_data.shape)
print("recent_data:", recent_data)

# 내일 종가를 예측해본다
test_predict = sess.run(hypothesis, feed_dict={X: recent_data})

test_predict_2 = []

for i in range(len(test_predict[0])):
    test_predict[0][i] = int(test_predict[0][i])
    test_predict_2.append((test_predict[0][i], i))

#final_predict = np.sort(test_predict_2, axis=1)[::-1]
print("test_predict_2  00000 =", test_predict_2)
test_predict_2.sort(key = lambda element : element[0], reverse=True)


# predict_six = []
#
# for i in final_predict:
#     for j in test_predict[0]:
#         if (final_predict[i] == test_predict[0][j])


#print("test_predict 0 ", test_predict[0])
print("test_predict_2  11111", test_predict_2)

picked = []
for i in range(6):
    picked.append(test_predict_2[i][1] + 1)

print(picked)

In [ ]:
# 내일 종가를 예측해본다
test_predict = sess.run(hypothesis, feed_dict={X: recent_data})

test_predict_2 = []

for i in range(len(test_predict[0])):
    test_predict[0][i] = int(test_predict[0][i])
    test_predict_2.append((test_predict[0][i], i))

#final_predict = np.sort(test_predict_2, axis=1)[::-1]
print("test_predict_2  00000 =", test_predict_2)
test_predict_2.sort(key = lambda element : element[0], reverse=True)


# predict_six = []
#
# for i in final_predict:
#     for j in test_predict[0]:
#         if (final_predict[i] == test_predict[0][j])


#print("test_predict 0 ", test_predict[0])
print("test_predict_2  11111", test_predict_2)

picked = []
for i in range(6):
    picked.append(test_predict_2[i][1] + 1)

print(picked)

test_predict_2  00000 = [(-28.0, 0), (9.0, 1), (7.0, 2), (3.0, 3), (31.0, 4), (10.0, 5), (40.0, 6), (9.0, 7), (8.0, 8), (15.0, 9), (6.0, 10), (11.0, 11), (24.0, 12), (28.0, 13), (38.0, 14), (17.0, 15), (20.0, 16), (1.0, 17), (55.0, 18), (-1.0, 19), (-9.0, 20), (4.0, 21), (0.0, 22), (0.0, 23), (-4.0, 24), (39.0, 25), (-16.0, 26), (18.0, 27), (-2.0, 28), (20.0, 29), (23.0, 30), (4.0, 31), (-20.0, 32), (-2.0, 33), (0.0, 34), (24.0, 35), (34.0, 36), (35.0, 37), (-4.0, 38), (8.0, 39), (12.0, 40), (29.0, 41), (71.0, 42), (2.0, 43), (22.0, 44)]
test_predict_2  11111 [(71.0, 42), (55.0, 18), (40.0, 6), (39.0, 25), (38.0, 14), (35.0, 37), (34.0, 36), (31.0, 4), (29.0, 41), (28.0, 13), (24.0, 12), (24.0, 35), (23.0, 30), (22.0, 44), (20.0, 16), (20.0, 29), (18.0, 27), (17.0, 15), (15.0, 9), (12.0, 40), (11.0, 11), (10.0, 5), (9.0, 1), (9.0, 7), (8.0, 8), (8.0, 39), (7.0, 2), (6.0, 10), (4.0, 21), (4.0, 31), (3.0, 3), (2.0, 43), (1.0, 17), (0.0, 22), (0.0, 23), (0.0, 34), (-1.0, 19), (-2.0, 28), 

# ALL Together

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1

main_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin" # 마지막 회차를 얻기 위한 주소
basic_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin&drwNo=" # 임의의 회차를 얻기 위한 주소

# 마지막 회차 정보를 가져옴
def GetLast(): 
    resp = requests.get(main_url)
    soup = BeautifulSoup(resp.text, "lxml")
    result = str(soup.find("meta", {"id" : "desc", "name" : "description"})['content'])
    s_idx = result.find(" ")
    e_idx = result.find("회")
    return int(result[s_idx + 1 : e_idx])

# 지정된 파일에 지정된 범위의 회차 정보를 기록함
def Crawler(s_count, e_count, fp):
    for i in range(s_count , e_count + 1):
        crawler_url = basic_url + str(i)
        resp = requests.get(crawler_url)
        soup = BeautifulSoup(resp.text, "html.parser")

        text = soup.text

        s_idx = text.find(" 당첨결과")
        s_idx = text.find("당첨번호", s_idx) + 4
        e_idx = text.find("보너스", s_idx)
        numbers = text[s_idx:e_idx].strip().split()

        s_idx = e_idx + 3
        e_idx = s_idx + 3
        bonus = text[s_idx:e_idx].strip()

        s_idx = text.find("1등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money1 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("2등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money2 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("3등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money3 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("4등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money4 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("5등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money5 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        line = str(i) + ',' + numbers[0] + ',' + numbers[1] + ',' + numbers[2] + ',' + numbers[3] + ',' + numbers[4] + ',' + numbers[5] + ',' + bonus + ',' + money1 + ',' + money2 + ',' + money3 + ',' + money4 + ',' + money5
        print(line)
        line += '\n'
        fp.write(line)

last = GetLast() # 마지막 회차를 가져옴

lotto_dir_name='lotto'
drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
if not path.exists(lotto_base_dir):
  print('Check your google drive directory. See you file explorer')


print ("Last=",last)

# 완전히 다시 쓰기
#fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'w')

# 기존 데이타에 추가하기
fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'a')
Crawler(last-1, last, fp) # 처음부터 마지막 회차까지 저장
fp.close()

#  --------------
#  번호 저장 끝
#  -------------

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers


rows = np.loadtxt("./gdrive/MyDrive/lotto_data.txt", delimiter=",")
row_count = len(rows)

numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

model = tf.keras.models.load_model('./gdrive/MyDrive/my_model.h5')
model.summary()

train_loss = []
train_acc = []
val_loss = []
val_acc = []


# 88회부터 지금까지 1등부터 5등까지 상금의 평균낸다.
mean_prize = [  np.mean(rows[87:, 8]),
            np.mean(rows[87:, 9]),
            np.mean(rows[87:, 10]),
            np.mean(rows[87:, 11]),
            np.mean(rows[87:, 12])]

train_total_reward = []
train_total_grade = np.zeros(6, dtype=int)

val_total_reward = []
val_total_grade = np.zeros(6, dtype=int)

test_total_reward = []
test_total_grade = np.zeros(6, dtype=int)



def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 300 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

   
    selected_balls.sort()

    return selected_balls

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []




# 최대 100번 에포크까지 수행
for epoch in range(50):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    batch_train_loss = []
    batch_train_acc = []

    for i in range(len(x_samples)):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))  


# Trainning 끝나면 무조건 모델에 저장
model.save('my_model.h5')
!cp /content/my_model.h5 /content/drive/My\ Drive/


# 마지막 회차까지 학습한 모델로 다음 회차 추론

# 50 개 뽑기
drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "predict2.txt"), "w") as f:
  print ("predict.txt")

  for n in range(50):
    numbers = gen_numbers_from_probability(ys_pred[0])  
    print('{0} : {1}'.format(n, numbers))    
    list_numbers.append(numbers)  
    line_str=','.join(str(e) for e in numbers)
    line_str += '\n'
    print(line_str)
    f.write(line_str)
f.close()

from ftplib import FTP

ftp = FTP('112.175.184.78')
ftp.login('dalasjoe', 'Dalasjoe75!')

# ftp.cwd('html') # "test"디렉터리로 이동
# ftp.retrlines('LIST') # 디렉터리의 내용을 목록화
# #ftp.retrbinary('RETR README', open('README', 'wb').write) # README 파일 저장
# ftp.quit()


ftp.cwd('html')  # 업로드할 FTP 폴더로 이동
myfile = open(path.join(lotto_base_dir, "predict2.txt"),'rb')  # 로컬 파일 열기
ftp.storbinary('STOR ' + 'predict2.txt', myfile )  # 파일을 FTP로 업로드
myfile.close()  # 파일 닫기

print ("File Saved")

     |████████████████████████████████| 380.5 MB 10 kB/s 
     |████████████████████████████████| 501 kB 47.8 MB/s 
     |████████████████████████████████| 4.3 MB 32.5 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 


Mounted at /content/gdrive
Last= 980
979,7,11,16,21,27,33,24,1606400518,47446429,1097911,50000,5000
980,3,13,16,23,24,35,14,3409443215,56824054,1411027,50000,5000
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 128)                  89088     
_________________________________________________________________
dense (Dense)                (1, 45)                   5805      
Total params: 94,893
Trainable params: 94,893
Non-trainable params: 0
_________________________________________________________________
receive numbers
epoch    0 train acc 0.171 loss 0.048
epoch    1 train acc 0.182 loss 0.048
epoch    2 train acc 0.195 loss 0.030
epoch    3 train acc 0.171 loss 0.025
epoch    4 train acc 0.192 loss 0.020
epoch    5 train acc 0.204 loss 0.017
epoch    6 train acc 0.172 loss 0.018
epoch    7 train acc 0.160 loss 0.018
epoch    8 train acc 0.195 loss

# 저장된 데이터 확인

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
import numpy as np
from google.colab import drive

otto_dir_name='lotto'
drive.mount('/content/gdrive',force_remount=True)

if not path.exists(lotto_base_dir):
  print('Check your google drive directory. See you file explorer')

lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "lotto_data.txt"), "r") as f:
  print("11111")
  line = f.read()
  print(">>>" + line)


# **데이터 수집**  - Crawling

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

main_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin" # 마지막 회차를 얻기 위한 주소
basic_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin&drwNo=" # 임의의 회차를 얻기 위한 주소

# 마지막 회차 정보를 가져옴
def GetLast(): 
    resp = requests.get(main_url)
    soup = BeautifulSoup(resp.text, "lxml")
    result = str(soup.find("meta", {"id" : "desc", "name" : "description"})['content'])
    s_idx = result.find(" ")
    e_idx = result.find("회")
    return int(result[s_idx + 1 : e_idx])

# 지정된 파일에 지정된 범위의 회차 정보를 기록함
def Crawler(s_count, e_count, fp):
    for i in range(s_count , e_count + 1):
        crawler_url = basic_url + str(i)
        resp = requests.get(crawler_url)
        soup = BeautifulSoup(resp.text, "html.parser")

        text = soup.text

        s_idx = text.find(" 당첨결과")
        s_idx = text.find("당첨번호", s_idx) + 4
        e_idx = text.find("보너스", s_idx)
        numbers = text[s_idx:e_idx].strip().split()

        s_idx = e_idx + 3
        e_idx = s_idx + 3
        bonus = text[s_idx:e_idx].strip()

        s_idx = text.find("1등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money1 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("2등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money2 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("3등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money3 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("4등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money4 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("5등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money5 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        line = str(i) + ',' + numbers[0] + ',' + numbers[1] + ',' + numbers[2] + ',' + numbers[3] + ',' + numbers[4] + ',' + numbers[5] + ',' + bonus + ',' + money1 + ',' + money2 + ',' + money3 + ',' + money4 + ',' + money5
        print(line)
        line += '\n'
        fp.write(line)

last = GetLast() # 마지막 회차를 가져옴

lotto_dir_name='lotto'
drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
if not path.exists(lotto_base_dir):
  print('Check your google drive directory. See you file explorer')

#with open(path.join(lotto_base_dir, "lotto_data.txt"), "r") as f:
#  print("11111")
#   line = f.read()
#   print(">>>" + line)

print ("Last=",last)

# 완전히 다시 쓰기
#fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'w')

# 기존 데이타에 추가하기
fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'a')

Crawler(1, last, fp) # 처음부터 마지막 회차까지 저장
fp.close()


## 데이터 로딩

In [ ]:
from os import path
from google.colab import drive
import numpy as np

drive.mount('/content/drive')

rows = np.loadtxt("./drive/MyDrive/lotto_data.txt", delimiter=",")
row_count = len(rows)

print("Data Loaded : row count: " + str(row_count))

Mounted at /content/drive
Data Loaded : row count: 965


# **TensorFlow 설치 및 모델 초기화**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers


numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

# 데이터셋 구성
train_idx = (0, 700) # 훈련셋 
val_idx = (700, 800) # 검증셋
test_idx = (800, len(x_samples)) #시험셋

print("train: {0}, val: {1}, test: {2}".format(train_idx, val_idx, test_idx))

# 모델을 정의합니다. - LSTM
#타입스텝은 1인 대신, 상태유지(stateful 옵션을 True)으로 설정했습니다.
#45개의 벡터로 출력합니다.
#각각의 벡터는 0.0과 1.0사이의 실수값으로 나옵니다. 각 벡터가 독립적으로 모두 1.0이 나오거나 0.0이 나올 수 있는 멀티레이블 문제입니다.
#멀티레이블 문제라 출력층의 활성화함수가 softmax가 아닌 sigmoid로 설정하였습니다.
model = keras.Sequential([
    keras.layers.LSTM(128, batch_input_shape=(1, 1, 45), return_sequences=False, stateful=True),
    keras.layers.Dense(45, activation='sigmoid')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 매 에포크마다 훈련과 검증의 손실 및 정확도를 기록하기 위한 변수
train_loss = []
train_acc = []
val_loss = []
val_acc = []


# 88회부터 지금까지 1등부터 5등까지 상금의 평균낸다.
mean_prize = [  np.mean(rows[87:, 8]),
            np.mean(rows[87:, 9]),
            np.mean(rows[87:, 10]),
            np.mean(rows[87:, 11]),
            np.mean(rows[87:, 12])]

print(mean_prize)   

train_total_reward = []
train_total_grade = np.zeros(6, dtype=int)

val_total_reward = []
val_total_grade = np.zeros(6, dtype=int)

test_total_reward = []
test_total_grade = np.zeros(6, dtype=int)

model.reset_states()

# print('[No. ] 1st 2nd 3rd 4th 5th 6th Rewards')

# for i in range(len(x_samples)):
#     xs = x_samples[i].reshape(1, 1, 45)
#     ys_pred = model.predict_on_batch(xs) # 모델의 출력값을 얻음
    
#     sum_reward = 0
#     sum_grade = np.zeros(6, dtype=int) # 6등까지 변수

#     for n in range(10): # 10판 수행
#         numbers = gen_numbers_from_probability(ys_pred[0])
        
#         #i회차 입력 후 나온 출력을 i+1회차와 비교함
#         grade, reward = calc_reward(rows[i+1,1:7], rows[i+1,7], numbers) 

#         sum_reward += reward
#         sum_grade[grade] += 1

#         if i >= train_idx[0] and i < train_idx[1]:
#             train_total_grade[grade] += 1
#         elif i >= val_idx[0] and i < val_idx[1]:
#             val_total_grade[grade] += 1
#         elif i >= test_idx[0] and i < test_idx[1]:
#             val_total_grade[grade] += 1
    
#     if i >= train_idx[0] and i < train_idx[1]:
#         train_total_reward.append(sum_reward)
#     elif i >= val_idx[0] and i < val_idx[1]:
#         val_total_reward.append(sum_reward)
#     elif i >= test_idx[0] and i < test_idx[1]:
#         test_total_reward.append(sum_reward)
                        
#     print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d} {7:15,d}'.format(i+1, sum_grade[0], sum_grade[1], sum_grade[2], sum_grade[3], sum_grade[4], sum_grade[5], int(sum_reward)))

# print('Total') 
# print('==========')    
# print('Train {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(train_total_grade[0], train_total_grade[1], train_total_grade[2], train_total_grade[3], train_total_grade[4], train_total_grade[5], int(sum(train_total_reward))))
# print('Val   {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(val_total_grade[0], val_total_grade[1], val_total_grade[2], val_total_grade[3], val_total_grade[4], val_total_grade[5], int(sum(val_total_reward))))
# print('Test  {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(test_total_grade[0], test_total_grade[1], test_total_grade[2], test_total_grade[3], test_total_grade[4], test_total_grade[5], int(sum(test_total_reward))))
# print('==========')   

In [ ]:
model = tf.keras.models.load_model('./drive/MyDrive/my_model.h5')
model.summary()

# **Trainning**

In [ ]:
# 최대 100번 에포크까지 수행

# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지"); 
#     document.querySelector("colab-toolbar-button#connect").click() 
# }
# setInterval(ClickConnect, 60 * 1000)

model = tf.keras.models.load_model('./drive/MyDrive/my_model.h5')
model.summary()

for epoch in range(50):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    batch_train_loss = []
    batch_train_acc = []

    for i in range(len(x_samples)):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))  


# Trainning 끝나면 무조건 모델에 저장
model.save('my_model.h5')
!cp /content/my_model.h5 /content/drive/My\ Drive/

# 마지막 회차까지 학습한 모델로 다음 회차 추론

 # 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

   
    selected_balls.sort()

    return selected_balls

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

# 50 개 뽑기
# for n in range(100):
#     numbers = gen_numbers_from_probability(ys_pred[0])  
#     print('{0} : {1}'.format(n, numbers))    
#     list_numbers.append(numbers)  


drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "predict.txt"), "w") as f:
  print ("predict.txt")

  for n in range(50):
    numbers = gen_numbers_from_probability(ys_pred[0])  
    print('{0} : {1}'.format(n, numbers))    
    list_numbers.append(numbers)  
    line_str=','.join(str(e) for e in numbers)
    line_str += '\n'
    print(line_str)
    f.write(line_str)
f.close()

OSError: ignored

# **Prediction**

In [ ]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론

 # 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

   
    selected_balls.sort()

    return selected_balls

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

# 50 개 뽑기
# for n in range(100):
#     numbers = gen_numbers_from_probability(ys_pred[0])  
#     print('{0} : {1}'.format(n, numbers))    
#     list_numbers.append(numbers)  


drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "predict.txt"), "w") as f:
  print ("predict.txt")

  for n in range(50):
    numbers = gen_numbers_from_probability(ys_pred[0])  
    print('{0} : {1}'.format(n, numbers))    
    list_numbers.append(numbers)  
    line_str=','.join(str(e) for e in numbers)
    line_str += '\n'
    print(line_str)
    f.write(line_str)
f.close()

# **Save Model**

In [ ]:
model.save('my_model.h5')

# **Load Model**

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1

drive.mount('/content/gdrive',force_remount=True)

new_model = tf.keras.models.load_model('./gdrive/MyDrive/my_model.h5')
new_model.summary()

Mounted at /content/gdrive
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 128)                  89088     
_________________________________________________________________
dense (Dense)                (1, 45)                   5805      
Total params: 94,893
Trainable params: 94,893
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!cp /content/my_model.h5 /content/drive/My\ Drive/

cp: failed to access '/content/drive/My Drive/': Transport endpoint is not connected


# FTP 전송

In [ ]:
from ftplib import FTP

ftp = FTP('112.175.184.78')
ftp.login('dalasjoe', 'Dalasjoe75!')

# ftp.cwd('html') # "test"디렉터리로 이동
# ftp.retrlines('LIST') # 디렉터리의 내용을 목록화
# #ftp.retrbinary('RETR README', open('README', 'wb').write) # README 파일 저장
# ftp.quit()


ftp.cwd('html')  # 업로드할 FTP 폴더로 이동
myfile = open(path.join(lotto_base_dir, "predict.txt"),'rb')  # 로컬 파일 열기
ftp.storbinary('STOR ' + 'predict.txt', myfile )  # 파일을 FTP로 업로드
myfile.close()  # 파일 닫기

print ("File Saved")

File Saved
